# Renamed PsExec
Detects the execution of a renamed PsExec often used by attackers or malware

## Rule Content
```
- title: Renamed PsExec
  id: a7a7e0e5-1d57-49df-9c58-9fe5bc0346a2
  status: experimental
  description: Detects the execution of a renamed PsExec often used by attackers or
    malware
  references:
  - https://www.trendmicro.com/vinfo/hk-en/security/news/cybercrime-and-digital-threats/megacortex-ransomware-spotted-attacking-enterprise-networks
  author: Florian Roth
  date: 2019/05/21
  tags:
  - car.2013-05-009
  logsource:
    product: windows
    service: sysmon
    category: null
  detection:
    selection:
      Description: Execute processes remotely
      Product: Sysinternals PsExec
    filter:
      Image:
      - '*\PsExec.exe'
      - '*\PsExec64.exe'
    condition: selection and not filter
  falsepositives:
  - Software that illegaly integrates PsExec in a renamed form
  - Administrators that have renamed PsExec and no one knows why
  level: high

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-endpoint-winevent-sysmon-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='((file_description:"Execute\ processes\ remotely" AND file_product:"Sysinternals\ PsExec") AND (NOT (process_path.keyword:(*\\PsExec.exe OR *\\PsExec64.exe))))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()